In [1]:
import json

import pvl
import requests

## Create the necessary input data

To create an ISD we require the following:

- target_name, e.g. Mercury
- capture_date (PDS data product dates appear to be ISO 8601 compliant)
- instrument, e.g. mdis-nac
- focal_plane_temperature (optionally included for temperature dependent focal lengths)
- spacecraft_id, e.g. messenger
- spacecraft_clock_count, from the label
- exposure_duration, from the label
- lighttime_correction, to tell spice what to use
- min_elevation (make optional?)
- max_elevation (make optional?)

For the last two, it might be easier/better to take the intital center point from the PDS label and hit a low res. DEM to find these automatically for the user.

In [2]:
#label = pvl.load('/data/usgs_csm/CSM-SET/notebooks/J03_046060_1986_XN_18N282W.IMG')
#print(label)
label = pvl.load('/data/messenger_sample/EN0211587012M.IMG')
data = {'target_name': label['TARGET_NAME'],
        'capture_date': label['START_TIME'].isoformat(),
        'instrument': label['INSTRUMENT_NAME'], 
        'focal_plane_temperature': label['FOCAL_PLANE_TEMPERATURE'].value,
        'spacecraft_id' : label['INSTRUMENT_HOST_NAME'],
        'spacecraft_clock_count': label['SPACECRAFT_CLOCK_START_COUNT'],
        'exposure_duration': label['EXPOSURE_DURATION'].value,
        'lighttime_correction':'LT+S', 
        'min_elevation': -100,
        'max_elevation': 100
        }
import json
data = json.dumps(data)

## Standard API landing page

In [3]:
r = requests.get('http://smalls:8002')
r.json()

{'apis': {'1.0': 'api/1.0'}, 'status': 'success'}

## Pushing the API into a namespace and versioning is good practice

In [4]:
r = requests.get('http://smalls:8002/api/1.0')
r.json()

{'data': {'available_missions': {'mercury': {'messenger': '/api/1.0/mercury/messenger'}}},
 'success': True}

## Example listing of the kernels

In [5]:
r = requests.get('http://smalls:8002/api/1.0/missions')
r.json()

{'data': {'meta_kernels': [{'mission': 'mars_reconnaissance_oribter'},
   {'mission': 'messenger'}]},
 'success': True}

In [3]:
r = requests.get('http://smalls:8002/api/1.0/missions/mars_reconnaissance_orbiter')
r.json()

{'data': {'kernels': {'metakernels': '/api/1.0/missions/mars_reconnaissance_orbiter/metakernels'}},
 'success': True}

## Example generation of the ISD object

Notice that every endpoint returns with a `success` key and a `data` key at the top level.  This helps the user know if the call was successful and that all of the response is within the `data` key.

In [14]:
import datetime
import json
import numpy as np
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, datetime.date):
            return obj.isoformat()
        return json.JSONEncoder.default(self, obj)

    
label = pvl.load('/data/usgs_csm/CSM-SET/notebooks/J03_046060_1986_XN_18N282W.IMG')
# Homogenize the data keys
label['IMAGE']['SAMPLES'] = label['IMAGE']['LINE_SAMPLES']

# Since we want to pass the 'raw' label, we must serialize it
data = json.dumps(label, cls=NumpyEncoder)

print(data)

{"PDS_VERSION_ID": "PDS3", "FILE_NAME": "J03_046060_1986_XN_18N282W.IMG", "RECORD_TYPE": "FIXED_LENGTH", "RECORD_BYTES": 5056, "FILE_RECORDS": 18433, "LABEL_RECORDS": 1, "^IMAGE": 2, "SPACECRAFT_NAME": "MARS_RECONNAISSANCE_ORBITER", "INSTRUMENT_NAME": "CONTEXT CAMERA", "INSTRUMENT_HOST_NAME": "MARS RECONNAISSANCE ORBITER", "MISSION_PHASE_NAME": "ESP", "TARGET_NAME": "MARS", "INSTRUMENT_ID": "CTX", "PRODUCER_ID": "MRO_CTX_TEAM", "DATA_SET_ID": "MRO-M-CTX-2-EDR-L0-V1.0", "PRODUCT_CREATION_TIME": "2016-10-04T21:05:58", "SOFTWARE_NAME": "makepds05 $Revision: 1.16 $", "UPLOAD_ID": "UNK", "ORIGINAL_PRODUCT_ID": "4A_04_10E803E100", "PRODUCT_ID": "J03_046060_1986_XN_18N282W", "START_TIME": "2016-05-24T11:51:26.043000", "STOP_TIME": "2016-05-24T11:52:00.636000", "SPACECRAFT_CLOCK_START_COUNT": "1148557929:105", "SPACECRAFT_CLOCK_STOP_COUNT": "N/A", "FOCAL_PLANE_TEMPERATURE": [295.8, "K"], "SAMPLE_BIT_MODE_ID": "SQROOT", "OFFSET_MODE_ID": "197/230/219", "LINE_EXPOSURE_DURATION": [1.877, "MSEC"],

In [15]:
r = requests.post('http://smalls:8002/api/1.0/missions/mars_reconnaissance_orbiter/csm_isd', data=data)
r.json()

{'data': {'isd': '{"DT_EPHEM": 0.15016, "NUMBER_OF_EPHEM": 231, "T0_QUAT": -17.298431992530823, "MIN_VALID_HT": -8000, "SEMI_MAJOR_AXIS": 3396190.0, "QUATERNIONS": [0.44952472860454074, -0.650326767154696, 0.2332535013479794, 0.5662114608554338, 0.4495527843900355, -0.6503571799691237, 0.2332112057591029, 0.5661716753937452, 0.4495807944710761, -0.6503871990905886, 0.2331684885810699, 0.566132543168713, 0.4496083504336671, -0.6504170958981307, 0.23312591474611905, 0.5660938442144942, 0.44963532207843643, -0.6504470599936377, 0.23308368467835094, 0.5660553817620824, 0.4496615122268406, -0.6504775228895638, 0.2330421128059219, 0.5660166872948886, 0.4496886638761759, -0.6505075510892894, 0.2330007639414586, 0.5659776281442331, 0.4497170872152782, -0.6505372878183401, 0.23295970468270547, 0.565937765676325, 0.44974591414553106, -0.650566941001349, 0.2329188583366568, 0.5658975820912265, 0.44977210359838066, -0.650597101347536, 0.23287794030248082, 0.565858932476413, 0.44979985565551067, -0

## Use the ISD

In [4]:
import usgscam as cam
from cycsm.isd import Isd

In [5]:
isd = r.json()['data']['isd']

In [6]:
i = Isd.loads(isd)

In [7]:
plugin = cam.genericls.GenericLsPlugin()
camera = plugin.from_isd(i, plugin.modelname(0))

In [13]:
camera.imagesize

(18432.0, 5000.0)

In [14]:
xyz = camera.imageToGround(9126, 2500, 0)
xyz

[701107.7192418213, 3142968.175070605, 1072678.7253195439]

In [15]:
camera.groundToImage(*xyz)

[9126.000016312359, 2500.000000206428]

## Random CTX Image

The above could be perceived as being too canned.  Here, pull a random image from the PDS and instantiate a camera model.

In [16]:
from ftplib import FTP
import os
from random import choice

In [41]:
# Run only once to avoid too many open FTP connections
ftp = FTP('pdsimage2.wr.usgs.gov')
ftp.login()
dirs = ftp.nlst('archive/mro-m-ctx-2-edr-l0-v1.0/')

In [42]:
data_dir = choice(dirs)
f = choice(ftp.nlst(data_dir + '/data'))
fname = os.path.basename(f)
with open('/data/ctx_sample/{}'.format(fname), 'wb') as file:
    ftp.retrbinary('RETR {}'.format(f), file.write)

In [43]:
newfile = os.path.join('/data/ctx_sample/{}'.format(fname))
newfile

'/data/ctx_sample/F04_037269_2011_XI_21N056W.IMG'

In [34]:
# Utility Func for working with PVL
def find_in_dict(obj, key):
    """
    Recursively find an entry in a dictionary

    Parameters
    ----------
    obj : dict
          The dictionary to search
    key : str
          The key to find in the dictionary

    Returns
    -------
    item : obj
           The value from the dictionary
    """
    if key in obj:
        return obj[key]
    for k, v in obj.items():
        if isinstance(v,dict):
            item = find_in_dict(v, key)
            if item is not None:
                return item
            
def data_from_cube(header):
    data = {}
    data['START_TIME'] = find_in_dict(header, 'StartTime')
    data['SPACECRAFT_NAME'] = find_in_dict(header, 'SpacecraftName')
    data['INSTRUMENT_NAME'] = find_in_dict(header, 'InstrumentId')
    data['SAMPLING_FACTOR'] = find_in_dict(header, 'SpatialSumming')
    data['SAMPLE_FIRST_PIXEL'] = find_in_dict(header, 'SampleFirstPixel')
    data['IMAGE'] = {}
    data['IMAGE']['LINES'] = find_in_dict(header, 'Lines')
    data['TARGET_NAME'] = find_in_dict(header, 'TargetName')
    data['LINE_EXPOSURE_DURATION'] = find_in_dict(header, 'LineExposureDuration')
    data['SPACECRAFT_CLOCK_START_COUNT'] = find_in_dict(header, 'SpacecraftClockCount')
    return data

In [35]:
import os

# Simple helper function to use the API
def create_data_pacakge(in_file):
    label = pvl.load(in_file)
    return json.dumps(label, cls=NumpyEncoder)

#newfile = '/data/ctx_sample/F04_037269_2011_XI_21N056W.IMG'
newfile = '/data/CTX/calibrated/B01_010045_1878_XN_07N205W.cal.cub'

print('Processing a new image: {}'.format(newfile))
# Call the func to create the data package for submission to the micro-service
data = pvl.load(newfile)
if 'cub' in os.path.splitext(newfile)[1]:
    data = data_from_cube(data)    
data = json.dumps(data, cls=NumpyEncoder)

print(data)
# Call the micro-service
r = requests.post('http://localhost:5000/api/1.0/missions/mars_reconnaissance_oribter/ctx/csm_isd', data=data)

# Get the ISD back and instantiate a local ISD for the image
isd = r.json()['data']['isd']
i = Isd.loads(isd)

# Create the plugin and camera as usual
plugin = cam.genericls.GenericLsPlugin()
camera = plugin.from_isd(i, plugin.modelname(0))

shape = camera.imagesize
# Call I2G
gnd = camera.imageToGround(shape[0] / 2, shape[1] / 2, 0)
print(gnd)

# Call G2I
camera.groundToImage(*gnd)

Processing a new image: /data/CTX/calibrated/B01_010045_1878_XN_07N205W.cal.cub
{"START_TIME": "2008-09-17T05:08:10.820000", "SPACECRAFT_NAME": "Mars_Reconnaissance_Orbiter", "INSTRUMENT_NAME": "CTX", "SAMPLING_FACTOR": 1, "SAMPLE_FIRST_PIXEL": 0, "IMAGE": {"LINES": 7168}, "TARGET_NAME": "Mars", "LINE_EXPOSURE_DURATION": [1.877, "MSEC"], "SPACECRAFT_CLOCK_START_COUNT": "0906095311:038"}
(7168.0, 5000.0)
[-3035183.4483585223, 1452666.3622457734, 457215.05215266405]


[3584.000022561103, 2500.0000033572246]

In [12]:
import pyproj
from math import sqrt

semi_major = i.param("SEMI_MAJOR_AXIS")
semi_minor = semi_major * sqrt(1 - i.param("ECCENTRICITY")**2)

ecef = pyproj.Proj(proj='geocent', a=semi_major, b=semi_minor)
lla = pyproj.Proj(proj='longlat', a=semi_major, b=semi_minor)

gnd = camera.imageToGround(shape[0], shape[1], 0)


lons, lats, alts = pyproj.transform(ecef, lla, gnd[0], gnd[1], gnd[2])
lons, lats

(-56.62606257827925, 21.679360463340597)